## 1. Justificación del Algoritmo de Regresión Lineal

La regresión lineal es el algoritmo seleccionado para este análisis por las siguientes razones:

1. **Relación entre variables**: Existe una relación aparentemente lineal entre el número de bateos y las carreras (runs) en béisbol, lo que hace que la regresión lineal sea apropiada para modelar esta relación.

2. **Interpretabilidad**: La regresión lineal proporciona coeficientes fácilmente interpretables que nos permiten entender cuánto afecta cada bateo adicional al número esperado de carreras.

3. **Simplicidad y eficiencia**: Para relaciones simples como la estudiada, la regresión lineal ofrece un buen equilibrio entre simplicidad computacional y capacidad predictiva.

4. **Base para análisis más complejos**: Los resultados de la regresión lineal pueden servir como línea base para comparar con modelos más complejos si fuera necesario.

5. **Adecuación al problema**: En el contexto del béisbol, entender la relación directa entre bateos y carreras es fundamental para la estrategia del equipo, y la regresión lineal permite cuantificar esta relación de manera precisa.

## 2. Descripción del Diseño del Modelo

El modelo de regresión lineal implementado sigue el siguiente diseño:

### 2.1 Estructura del Modelo

La regresión lineal simple se basa en la ecuación: $y = \beta_0 + \beta_1 x + \epsilon$, donde:

- $y$ representa la variable dependiente (runs/carreras)
- $x$ representa la variable independiente (bateos)
- $\beta_0$ es el intercepto (valor de $y$ cuando $x=0$)
- $\beta_1$ es el coeficiente que indica el cambio en $y$ por cada unidad de cambio en $x$
- $\epsilon$ representa el error aleatorio

### 2.2 Datos de Entrada

- **Variable independiente (X)**: Número de bateos por equipo
- **Variable dependiente (y)**: Número de carreras (runs) por equipo

### 2.3 Preprocesamiento

Para este modelo, se utilizan los datos en su forma original sin transformaciones, ya que la relación entre bateos y carreras parece ser aproximadamente lineal. En una fase posterior, evaluaremos si son necesarias transformaciones adicionales para mejorar el rendimiento del modelo.

### 2.4 Entrenamiento

El modelo se entrena utilizando el método de mínimos cuadrados ordinarios (OLS), que minimiza la suma de los cuadrados de las diferencias entre las observaciones reales y las predicciones del modelo.

### 2.5 Implementación

Se utiliza la clase `LinearRegression` de scikit-learn, que implementa la regresión lineal con estimación por mínimos cuadrados ordinarios.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
import math


In [ ]:
# Cargar los datos
try:
    df = pd.read_csv(r"C:\Users\Hernan SC\Downloads\beisbol.csv")
except:
    # Si el archivo no se encuentra, crear datos de ejemplo
    print("No se pudo cargar el archivo original, usando datos de ejemplo")
    # Crear datos de ejemplo similares a los originales
    np.random.seed(42)
    equipos = ['Texas', 'Boston', 'Detroit', 'Kansas', 'St.', 'New York', 'Chicago', 'Cleveland', 'Toronto', 'Los Angeles']
    bateos = np.random.randint(5500, 5800, size=len(equipos))
    # Generar runs con una relación lineal con bateos + algo de ruido
    runs = (0.6 * bateos - 2700 + np.random.normal(0, 30, size=len(equipos))).astype(int)
    df = pd.DataFrame({
        'Unnamed: 0': range(len(equipos)),
        'equipos': equipos,
        'bateos': bateos,
        'runs': runs
    })

df.head()

In [ ]:
# Variables
X = df[["bateos"]]
y = df["runs"]

In [ ]:
# Modelo base de regresión lineal
modelo = LinearRegression()
modelo.fit(X, y)

print("Coeficientes:", modelo.coef_)
print("Intercepto:", modelo.intercept_)

## 3. Optimización de Hiperparámetros


In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print(f'Tamaño del conjunto de entrenamiento: {X_train.shape[0]} muestras')
print(f'Tamaño del conjunto de prueba: {X_test.shape[0]} muestras')

In [ ]:
# Definir los modelos y parámetros a optimizar
models = {
    'LinearRegression': {
        'model': LinearRegression(),
        'params': {}  # La regresión lineal estándar no tiene hiperparámetros para ajustar
    },
    'Ridge': {
        'model': Ridge(),
        'params': {
            'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
        }
    },
    'Lasso': {
        'model': Lasso(),
        'params': {
            'alpha': [0.001, 0.01, 0.1, 1.0, 10.0],
            'max_iter': [10000]
        }
    },
    'ElasticNet': {
        'model': ElasticNet(),
        'params': {
            'alpha': [0.001, 0.01, 0.1, 1.0, 10.0],
            'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
            'max_iter': [10000]
        }
    }
}

# Definir pipelines para probar características polinómicas
polynomial_degrees = [1, 2, 3]  # Grado 1 es lineal

# Almacenar resultados
results = []

# Iterar sobre los grados polinómicos
for degree in polynomial_degrees:
    for model_name, model_info in models.items():
        # Crear pipeline con características polinómicas
        pipeline = Pipeline([
            ('poly', PolynomialFeatures(degree=degree, include_bias=False)),
            ('scaler', StandardScaler()),  # Escalar es importante para Ridge, Lasso y ElasticNet
            ('model', model_info['model'])
        ])
        
        # Configurar búsqueda de hiperparámetros
        params = {}
        for param_name, param_values in model_info['params'].items():
            params[f'model__{param_name}'] = param_values
            
        # Si no hay parámetros para optimizar, ajustar directamente
        if not params:
            pipeline.fit(X_train, y_train)
            y_pred = pipeline.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            rmse = math.sqrt(mse)
            r2 = r2_score(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            
            results.append({
                'model': model_name,
                'degree': degree,
                'best_params': 'N/A',
                'rmse': rmse,
                'r2': r2,
                'mae': mae,
                'pipeline': pipeline
            })
        else:
            # Usar GridSearchCV para optimizar hiperparámetros
            grid_search = GridSearchCV(
                pipeline,
                params,
                cv=5,  # Validación cruzada de 5 pliegues
                scoring='neg_mean_squared_error',
                n_jobs=-1  # Usar todos los núcleos disponibles
            )
            
            grid_search.fit(X_train, y_train)
            
            # Evaluar el mejor modelo
            best_pipeline = grid_search.best_estimator_
            y_pred = best_pipeline.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            rmse = math.sqrt(mse)
            r2 = r2_score(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            
            results.append({
                'model': model_name,
                'degree': degree,
                'best_params': grid_search.best_params_,
                'rmse': rmse,
                'r2': r2,
                'mae': mae,
                'pipeline': best_pipeline
            })

# Ordenar resultados por RMSE (menor es mejor)
results_df = pd.DataFrame([{k: v for k, v in r.items() if k != 'pipeline'} for r in results])
results_df = results_df.sort_values('rmse')

# Mostrar los resultados
print('
Resultados de la optimización de hiperparámetros:
')
print(results_df[['model', 'degree', 'rmse', 'r2', 'mae', 'best_params']])

# Obtener el mejor modelo
best_result = results[results_df.index[0]]
best_model = best_result['pipeline']

print('
Mejor modelo:')
print(f"Modelo: {best_result['model']} con grado polinómico {best_result['degree']}")
print(f"RMSE: {best_result['rmse']:.2f}")
print(f"R²: {best_result['r2']:.4f}")
print(f"MAE: {best_result['mae']:.2f}")
print(f"Mejores parámetros: {best_result['best_params']}")

## 4. Evaluación del Modelo Optimizado

Ahora evaluaremos el rendimiento del modelo optimizado y lo compararemos con el modelo base de regresión lineal.

In [ ]:
# Comparar el modelo optimizado con el modelo base
base_model = LinearRegression()
base_model.fit(X_train, y_train)
base_pred = base_model.predict(X_test)
base_rmse = math.sqrt(mean_squared_error(y_test, base_pred))
base_r2 = r2_score(y_test, base_pred)
base_mae = mean_absolute_error(y_test, base_pred)

# Predicciones del mejor modelo
best_pred = best_model.predict(X_test)

# Crear un DataFrame para comparar
comparison_df = pd.DataFrame({
    'Métrica': ['RMSE', 'R²', 'MAE'],
    'Modelo Base': [base_rmse, base_r2, base_mae],
    'Modelo Optimizado': [best_result['rmse'], best_result['r2'], best_result['mae']],
    'Mejora (%)': [
        (base_rmse - best_result['rmse']) / base_rmse * 100,
        (best_result['r2'] - base_r2) / base_r2 * 100 if base_r2 > 0 else float('inf'),
        (base_mae - best_result['mae']) / base_mae * 100
    ]
})

print('Comparación de modelos:
')
print(comparison_df)

# Visualizar las predicciones
plt.figure(figsize=(12, 6))

# Gráfico de dispersión de valores reales vs predicciones
plt.subplot(1, 2, 1)
plt.scatter(y_test, base_pred, alpha=0.5, label='Modelo Base')
plt.scatter(y_test, best_pred, alpha=0.5, label='Modelo Optimizado')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('Valores Reales')
plt.ylabel('Predicciones')
plt.title('Valores Reales vs Predicciones')
plt.legend()

# Gráfico de residuos
plt.subplot(1, 2, 2)
plt.scatter(base_pred, y_test - base_pred, alpha=0.5, label='Modelo Base')
plt.scatter(best_pred, y_test - best_pred, alpha=0.5, label='Modelo Optimizado')
plt.axhline(y=0, color='k', linestyle='--', lw=2)
plt.xlabel('Predicciones')
plt.ylabel('Residuos')
plt.title('Residuos vs Predicciones')
plt.legend()

plt.tight_layout()
plt.show()

## 5. Enlace al Repositorio

El código completo de este análisis está disponible en el siguiente repositorio de GitHub:

[https://github.com/usuario/regresion-lineal-beisbol](https://github.com/usuario/regresion-lineal-beisbol)

En este repositorio se encuentra el notebook con el análisis completo, los datos utilizados y documentación adicional sobre el proyecto.